In [ ]:
# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np 
import pandas as pd
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.metrics import accuracy_score
from scipy import stats


import warnings
warnings.filterwarnings("ignore")

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train_fold= pd.read_csv("../input/tps-feb-2022-kfold/train_folds.csv") 
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
data = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [ ]:
targets = train_fold.target.unique()
target_dict = {t: i for i, t in enumerate(targets)} 
target_dict_inv = {v: k for k, v in target_dict.items()}
data["target_map"] = data.target.map(target_dict)

In [ ]:
train_split, val_split = train_test_split(data,test_size=0.25,random_state=42)
train_split.shape,val_split.shape

In [ ]:
train_split.head()

In [ ]:
val_split.head()

In [ ]:
features = [c for c in train_fold.columns if c not in ("target", "kfold","target_map")]
target="target_map"
df_test=test[features]

X_train= train_split[features]
y_train= train_split[target]
X_valid= val_split[features]
y_valid= val_split[target]
X_test= test[features]

In [ ]:
def objective(trial):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000, 100),
        'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    }
   
    model = XGBClassifier(random_state=42, 
                             tree_method='gpu_hist', 
                             gpu_id=0, 
                             predictor="gpu_predictor"
                             ,**param )  
    model.fit(X_train, y_train,eval_set=[(X_valid,y_valid)], early_stopping_rounds=150,verbose=False)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)
#study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# Visualizamos la importancia de los parámetros
optuna.visualization.plot_param_importances(study)

In [ ]:
####

In [ ]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train_fold= pd.read_csv("../input/tps-feb-2022-kfold/train_folds.csv") 
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [ ]:
targets = train_fold.target.unique()
target_dict = {t: i for i, t in enumerate(targets)} 
target_dict_inv = {v: k for k, v in target_dict.items()}
train_fold["target_map"] = train_fold.target.map(target_dict)
features = [c for c in train_fold.columns if c not in ("target", "kfold","target_map")]
target="target_map"
df_test=test[features]

In [ ]:
params = {'max_depth': 10, 'subsample': 0.7, 'n_estimators': 5100, 'eta': 0.09999999999999999, 'reg_alpha': 1, 'reg_lambda': 73, 'min_child_weight': 5, 'colsample_bytree': 0.8377366123284274}

In [ ]:
cv_preds_test = []
scores = []

for fold in range(7):
    X_train = train_fold[train_fold.kfold != fold].reset_index(drop=True)
    X_valid = train_fold[train_fold.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train=X_train[target]
    y_valid=X_valid[target]
    
    
    X_train=X_train[features]
    X_valid=X_valid[features]
    
    # Encoding e ingeniería de funciones
    ##
    ##
    #
    
    
    # Entrenamiento
    model = XGBClassifier(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor",**params)
    model.fit(X_train,y_train)
    preds_valid=model.predict(X_valid)
    test_preds = model.predict(X_test)
    cv_preds_test.append(test_preds)
    acc = accuracy_score(y_valid, preds_valid)
    print(fold, acc)
    scores.append(acc)
    
print(f"La media es {np.mean(scores)} y la desviación estándar es {np.std(scores)}")

In [ ]:
sample_sub['target'] =stats.mode(np.column_stack(cv_preds_test), axis=1)[0]
sample_sub['target']=sample_sub['target'].map(target_dict_inv)
sample_sub.head(10)
sample_sub.to_csv('submission.csv', index=False)